<a href="https://colab.research.google.com/github/tedblackson/DeepLearning/blob/main/DL_Lab_5_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch

#Dense Layer

In [8]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # record the inputs
    self.inputs = inputs
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases

  # Backward pass
  def backward(self, dvalues):
        self.dweights = torch.matmul(self.inputs.T, dvalues)
        self.dbiases = torch.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = torch.matmul(dvalues, self.weights.T)

#Relu Activation

In [9]:
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    # Remember input values
    self.inputs = inputs
    self.output = torch.max(torch.tensor(0),inputs)
  # Backward pass
  def backward(self, dvalues):
    self.dinputs = dvalues
    # Zero gradient where input values were negative
    self.dinputs[self.inputs <= 0] = 0

#Sigmoid Activation

In [10]:

class Activation_Sigmoid:
    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate sigmoid activation
        self.output = 1 / (1 + torch.exp(-inputs))

    # Backward pass
    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))


#Neural Network Architecture

In [11]:
class NeuralNetwork:
    def __init__(self, n_inputs, n_hidden, n_output):
        self.hidden_layer = DenseLayer(n_inputs, n_hidden)
        self.relu_activation = Activation_ReLU()

        self.output_layer = DenseLayer(n_hidden, n_output)
        self.sigmoid_activation = Activation_Sigmoid()

    def forward(self, inputs):
        self.hidden_layer.forward(inputs)
        self.relu_activation.forward(self.hidden_layer.output)

        self.output_layer.forward(self.relu_activation.output)
        self.sigmoid_activation.forward(self.output_layer.output)

    def backward(self, dvalues):
        self.sigmoid_activation.backward(dvalues)
        self.output_layer.backward(self.sigmoid_activation.dinputs)

        self.relu_activation.backward(self.output_layer.dinputs)
        self.hidden_layer.backward(self.relu_activation.dinputs)

# Example usage

In [15]:

nn = NeuralNetwork(n_inputs=2, n_hidden=2, n_output=2)

# Dummy data
inputs = torch.tensor([[0.5, 0.7]])
targets = torch.tensor([[1, 0]])

# Training loop
epochs = 10
for epoch in range(epochs):
    # Forward pass
    nn.forward(inputs)

    # Calculating the loss
    loss = torch.sum((nn.sigmoid_activation.output - targets) ** 2)

    # Backward pass to update gradients
    nn.backward(nn.sigmoid_activation.output - targets)

    # Update parameters
    learning_rate = 0.01
    nn.hidden_layer.weights -= learning_rate * nn.hidden_layer.dweights
    nn.hidden_layer.biases -= learning_rate * nn.hidden_layer.dbiases
    nn.output_layer.weights -= learning_rate * nn.output_layer.dweights
    nn.output_layer.biases -= learning_rate * nn.output_layer.dbiases


    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')


nn.forward(inputs)
print("Final Predictions:", nn.sigmoid_activation.output)

Epoch 1/10, Loss: 0.4999989867210388
Epoch 2/10, Loss: 0.49937403202056885
Epoch 3/10, Loss: 0.49874991178512573
Epoch 4/10, Loss: 0.49812668561935425
Epoch 5/10, Loss: 0.49750417470932007
Epoch 6/10, Loss: 0.4968824088573456
Epoch 7/10, Loss: 0.49626147747039795
Epoch 8/10, Loss: 0.49564129114151
Epoch 9/10, Loss: 0.495021790266037
Epoch 10/10, Loss: 0.49440324306488037
Final Predictions: tensor([[0.5032, 0.4969]])
